# Understanding the AWRA Community Modelling System package and its modules

   1. What is the AWRA CMS?
   2. AWRA-CMS package and modules
   3. AWRA CMS concepts and classes  




## 1.  What is the AWRA CMS?




   - Available to ***the community*** https://github.com/awracms/awra_cms/
   - Enables joint development and own application
   - ***User Guide***
       https://github.com/awracms/awra_cms/blob/master/docs/AWRA%20Landscape%20Community%20Modelling%20System%20User%20Guide%202019%20v3.2.docx
       
![title](awracms_schematic.PNG)


### How do I install and use the AWRA CMS?
To install the AWRA-CMS on your own system please see the ***INSTALL-GUIDE.md*** file on GitHub: https://github.com/awracms/awra_cms/blob/master/INSTALL-GUIDE.md. This install guide contains instructions for both Linux and Windows.
    
### Having issues with installing?
If you have any questions relating to install contact us at: awracms@bom.gov.au 

## 2. AWRA-CMS package and modules

#### There are 6 module components to the AWRAMS python package
* ***awrams.models***        : the AWRA-L model code
* ***awrams.simulation***    : tools to run the model 
    * **ondemand**: keeps the inputs and outputs stored in memory - for fast interactive use
    * **server**: direct to disk output, used for large scale runs (larger than available memory)  
* ***awrams.visualisation*** : contains functions to facilitate viewing of outputs and inputs across periods and spatial extents
* ***awrams.calibration***   : contains tools for calibration and evaluation of sensitivity to model parameters
* ***awrams.benchmarking***  : contains tools to allow comparison of multiple model outputs to observations
* ***awrams.utils***         : contains various support tools used throughout the rest of the system

In [ ]:
# import the modules
import awrams.models  # contains the awra model code
import awrams.simulation  # contains tools to run the model in memory or with written outputs
import awrams.visualisation # contains functions to facilitate viewing of outputs and inputs across periods and spatial extents
import awrams.calibration # contains tools for calibration and evaluation of sensitivity to model parameters
import awrams.benchmarking # contains tools to allow comparison of multiple model outputs to observations
import awrams.utils # contains tools to enable extraction of data from model outputs

In [ ]:
# see location of each of the components
awrams.models

#### There is a copy of the code provided in the awrams_cm folder to aid understanding
     
   - [Models folder]       :   ../../packages/awrams/models
   - [Simulation folder]   :   ../../packages/awrams/simulation
   - [Visualisation folder]:   ../../packages/awrams/visualisation
   - [Calibration folder]  :   ../../packages/awrams/calibration
   - [Benchmarking folder] :   ../../packages/awrams/benchmarking 

[Models folder]: ../../packages/awrams/models/
[Simulation folder]: ../../packages/awrams/simulation/
[Visualisation folder]: ../../packages/awrams/visualisation/
[Calibration folder]: ../../packages/awrams/calibration/
[Benchmarking folder]: ../../packages/awrams/benchmarking/

#### Use jupyter navigation to view/edit at copy of code:

  - For example the input tranforms file in the models folder [../../packages/awrams/models/awral/transforms.py]
 
[../../packages/awrams/models/awral/transforms.py]: ../../packages/awrams/models/awral/transforms.py

# 3. AWRA CMS concepts and classes 



## 3.1 Configuring AWRA simulations and calibrations

 - Setting up a simulation or calibration run starts by defining:

   - A.  The **model** (in this case AWRAL) :
   - B.  The inputs and outputs via **the nodegraph**
      1. Inputs:
         - the forcing inputs (rain, temperature, radiation) 
         - the spatial inputs (grids of various landscape properties)
         - model parameters (calibrated or fixed)
      2. Outputs:
         - the model outputs to be written out (e.g. flow, evaporation, soil moisture)
         - the grid resolution (default is 0.05 degree)
         
   - C.  The spatial **extent** 
   - D.  The modelling time **period**



## 3.1.A. Load the awral model class from the models module



#### *Model* class (awrams.models.model.Model): see Model class within [../../packages/awrams/models/awral/model.py]

- This Generic model class provides an interface for configuring and querying models.  
- All models must derive from this class.   
- Models conforming to this interface specification are used by various clients in AWRAMS:  
     - the **OnDemandSimulator** 
     - the **SimulationServer** and 
     - the AWRAMS **calibration** system.
- Models need to supply:
   - information about themselves (eg names of inputs, outputs and states), 
   - default input mapping (get_default_mapping())

[../../packages/awrams/models/awral/model.py]: ../../packages/awrams/models/awral/model.py

In [ ]:
### A. Load the awral model class from the models module
from awrams.models import awral

In [ ]:
# see the components of the awral class by tab completion (model, runner, settings, support, solar, template, transforms)
awral.model

In [ ]:
# Load model profile for awral model
from awrams.utils import config_manager
model_profile = config_manager.get_model_profile('awral', 'v6_default')
# Get default model settings from model_profile
model_settings = model_profile.get_settings()

In [ ]:
# Instantiate the AWRA-L model
awralmod = model_profile.get_model(model_settings)


### -----   Advanced Section:    Components of awrams.models.model.Model class -----------------------------------------------
#### Important components of awrams model class.  
   
#### [transforms.py] , [solar.py] 
[solar.py]: ../../packages/awrams/models/awral/solar.py
[transforms.py]: ../../packages/awrams/models/awral/transforms.py

- AWRA-L supplies a number of custom node types for use in its input mapping graph;  
   - read awral.model.Model.get_default_input_mapping() for particulars.
   
#### [template.py]
[template.py]: ../../packages/awrams/models/awral/template.py
- This file specifies how the awral_t.c is transformed into awral.c before compilation.   
- Modifying these templates is an advanced topic not covered here.

#### [runner.py]
[runner.py]: ../../packages/awrams/models/awral/runner.py
- This contains the ModelRunner implementation, and associated support code for C bindings, and should not need to be modified.
     
#### *ModelRunner* class (awrams.models.model.ModelRunner)
 - Hydrological processes should be represented in the ModelRunner.  
 - This class provides the interface for executing model runs
    - taking the outputs from an ExecutionGraph, and 
    - running them through the model’s hydrological processes.
 - Every Model needs to supply a runner via the model.get_runner() call. 

#### C code  and Dynamic model compilation

 - The input mapping/configuration of AWRA-L is pure Python
 - The core of AWRA-L is implemented in C for performance reasons using a template file
 
 
 - **Templating**: lightweight templating library is used to autogenerate some portions of code
 
     - The model template file (**[awral_t.c]**) is compiled automatically when required
     
               see ../../packages/awrams/models/awral/core/awral_t.c
 
     - Users wishing to edit the code should edit this file within the awra parackage
     - Recompilation occurs in particular when the mapping of input and output data changes
     
  - **Advantages**: 
     
     - Outputs can easily be added/removed, as well as input types changed (e.g. changing a scalar value to a spatial or timeseries forcing input), without users having to manually edit the C code
     - Does not recompile if not required


 

[awral_t.c]: ../../packages/awrams/models/awral/core/awral_t.c
  
### We'll come back to altering the code in the Advanced section later today.

###  -----   End Advanced: Components of awrams.models.model.Model class ------------------------------------------------------

## 3.1.B. Input/Output Mappings and Nodegraphs

 - ***Mappings*** are python dictionaries used to describe the data-flow
      - AWRAMS ***NodeGraph*** subsystem (awrams.utils.nodegraph) transforms this **configuration** info into runnable code.
 - Default Input mapping: the ‘data’ portion of a model
      - loading of files
      - infilling
      - unit conversion etc. 
 - Example:
     - AWRA-L model uses a single temperature input, but AWAP supplies two (minimum and maximum temperature).  
     - The default AWRA-L input mapping loads the AWAP inputs then rectifies and computes a weighted average of these, before passing the single value on to the core model code. 

### B.1 Input configuration

#### Default input static spatial datasets and model parameters:  
  See data folder [../../packages/awrams/models/awral/data/]


   - ***static spatial grids***:  various static soil, vegetation and topography related spatial datasets - see spatial_parameters.h5 (https://en.wikipedia.org/wiki/Hierarchical_Data_Format)
   - ***default model parameters***: see DefaultParameters.json (https://en.wikipedia.org/wiki/JSON)
   - ***model inputs***: complete list of input parameters (climate, static, spatial) - see model_inputs.json
   
for further details see [1.1_The_AWRA-L_model.ipynb]

[../../packages/awrams/models/awral/data/]: ../../packages/awrams/models/awral/data   
[1.1_The_AWRA-L_model.ipynb]: ../../Training/Basics/1.1_The_AWRA-L_model.ipynb

### Model settings - settings that are specific to the version of AWRA-L that is being used

In [ ]:
# Model settings
model_profile = config_manager.get_model_profile('awral', 'v6_default')
# Get default model settings from model_profile
model_settings = model_profile.get_settings()
model_settings

### System settings - settings that are specific to the system being used

In [ ]:
# The system profile contains non-model-specific information related to the system we are running on, like file paths
sys_profile = config_manager.get_system_profile()
sys_settings = sys_profile.get_settings()
sys_settings

### Model inputs

In [ ]:
# list the complete set of model inputs
model_inputs = awralmod.model_settings['CONFIG_OPTIONS']['MODEL_INPUTS']
model_inputs

### Need to map values to these AWRA-L input parameters

- See get_input_mapping() in model_profile
- Various [nodes.py] types are used to map input values to these parameters
    - climate input from netcdf files:  nodes.forcing_from_ncfiles(CLIMATE_DATA,v[0],v[1])
    - static spatial inputs from grids: nodes.spatial_from_file(SPATIAL_FILE,'parameters/%s' % grid)
    - assigning a constant value eg. air pressure mapping['pair'] = nodes.const(97500.)
    - transformations: eg.
         - average temperature mapping['tat'] = nodes.mix('tmin','tmax',0.75)
         - multiplying a spatial grid by an input parameter mapping['s0max'] = nodes.mul('s0max_scale','s0fracawc_grid',100.)


[nodes.py]: ../../../edit/utils/awrams/utils/nodegraph/nodes.py
[model.py]: ../../../edit/models/awrams/models/awral/model.py

In [ ]:
# View the input configuration mapping
input_mapping = model_profile.get_input_mapping()
input_mapping

In [ ]:
# We can explore the elements of the input mapping individually
# e.g.
input_mapping['precip_f']

In [ ]:
# e.g.
input_mapping['s0max']

In [ ]:
# Convenience function to allow you to inspect what goes into creating a parameter
from awrams.utils.nodegraph import nodes, graph
graph.get_input_tree(['k_rout'], input_mapping)

In [ ]:
# climate forcing mapping and path
awralmod.model_settings['CLIMATE_DATASET']['FORCING']

In [ ]:
# climate data path
sys_settings['CLIMATE_DATASETS']['TRAINING']['FORCING']['PATH']

In [ ]:
# view default data path TRAINING_DATA_PATH
TRAINING_DATA_PATH = sys_settings['DATA_PATHS']['TRAINING_DATA']
TRAINING_DATA_PATH

#### Input climate data is read by default according to the following pattern 

nc_var=path  path +  pattern

rain_day=   ./rain_day/rain*

#### Need to alter this to point at your input data: We demontrate this below..

### Datasets in AWRA CMS

On installation the AWRA-CMS data is obtained from (https://github.com/awracms/awracms_data) which contains data under the following folders:
  - benchmarking: 
  - climatology
  - model_data
  - observations
  - spatial
  - test_data
  - training

You can specify where this data is downloaded to during installation, or proceed with the default path of '../../awrams/data'.

### Input daily climate (forcing) data: 
Daily climate data required for running the AWRA-L model includes:

 - ***rainfall***
 - ***temperature*** (an average of maximum and minimum values)
 - ***solar radiation***
 - ***wind***

The Bureau of Meteorology uses 0.05 degree gridded daily data based on 9am-9am observations and satellite data across Australia; see http://www.bom.gov.au/jsp/awap/. This data covers 1911 until yesterday, with the exception of solar radiation based on satellite data; where climatologies are used prior to 1990. You can use your own data provided you can format it into netcdf input file format.

Limited data (2009-2011) is provided with the AWRA-CMS. If you wish to download a longer period (e.g. 1980-2018) of daily climate forcing data please contact us as awracms@bom.gov.au for assistance.

### Input starting states
The AWRA-L model have the following model states that change from one time-step to the next:

  - **mleaf** : vegetation mass (mm) - for deep rooted (dr) and shallow rooted (sr) HRUs
  - **s0**    : top layer 0-10cm soil moisture (mm)  - for deep rooted (dr) and shallow rooted (sr) HRUs
  - **ss**    : upper layer 10cm-100cm soil moisture (mm) - for deep rooted (dr) and shallow rooted (sr) HRUs
  - **sd**    : deep layer 100cm-600cm soil moisture (mm) - for deep rooted (dr) and shallow rooted (sr) HRUs
  - **sg**    : saturated groundwater storage (mm)
  - **sr**    : surface water storage (mm)
  
These states are saved as part of model simulation, for potential use in initialising model runs. These inital states are required to be specified when starting a simulation or calibration, if they are not set default values will be used.

In [ ]:
# list five variables folders: wind, rain_day   (rainfall), solar_exposure_day   (solar radiation), temp_max_day   (maximum temperature), temp_min_day (temperature minimum)
data_path = TRAINING_DATA_PATH + '/climate/bom_awap/'
!ls $data_path

### Alter location of input climate data to ../../test_data/simulation

In [ ]:
# Create a function change_path_to_forcing() to change from the default paths to  
def change_path_to_forcing(imap, path):
    from awrams.utils.nodegraph import nodes
    from os.path import join
    from os import getcwd

    data_path = TRAINING_DATA_PATH + path
    FORCING = {
        'tmin': ('temp_min*.nc','temp_min_day',data_path + 'temp_min_day/'),
        'tmax': ('temp_max*.nc','temp_max_day',data_path + 'temp_max_day/'),
        'precip': ('rain_day*.nc','rain_day',data_path + 'rain_day/'),
        'solar': ('solar*.nc','solar_exposure_day',data_path + 'solar_exposure_day/') ,
        'wind': ('wind*.nc','wind',data_path + 'wind/') #,
    }     
    
    # location of registered user data in the Training folder
    #data_path = '../test_data/climate/BOM_climate/'
    #FORCING = {
    #    'tmin': ('temp_min*.nc','temp_min_day',data_path + 'temp_min_day/'),
    #    'tmax': ('temp_max*.nc','temp_max_day',data_path + 'temp_max_day/'),
    #    'precip': ('rain_day*.nc','rain_day',data_path + 'rain_day/'),
    #    'solar': ('solar*.nc','solar_exposure_day',data_path + 'solar_exposure_day/') #,
    #}     
    
    for k,v in FORCING.items():
        imap[k+'_f'] = nodes.forcing_from_ncfiles(v[2],v[0],v[1])

# alter the input map by calling function above
change_path_to_forcing(input_mapping, path = '../test_data/simulation/climate/')

In [ ]:
# Path to Tmin data after change to default_config...
print('tmin_f',input_mapping['tmin_f'])

### B.2 Output configuration


In [ ]:
# default outputs
awralmod.get_output_mapping()

In [ ]:
# list types of outputs

# OUTPUTS_HRU= outputs on a HRU basis
# OUTPUTS_AVG= outputs on a cell basis averaged from HRU values 
awralmod.OUTPUTS.keys()

In [ ]:
# OUTPUTS_CELL= outputs on a cell/grid/catchment basis
awralmod.OUTPUTS['OUTPUTS_CELL'] # ['qtot', 'sr', 'sg'] = runoff, surface water storage, groundwater storage

In [ ]:
awralmod.OUTPUTS['OUTPUTS_HRU'] # ['s0', 'ss', 'sd', 'mleaf'] = top, shallow and seep soil moisture & leaf mass 

In [ ]:
awralmod.OUTPUTS['OUTPUTS_AVG'] # ['e0', 'etot', 'dd', 's0', 'ss', 'sd']= potential and actual ET, deep drainage,top, shallow and seep soil moisture

In [ ]:
# add some extra outputs via the output_mapping
awralmod.OUTPUTS['OUTPUTS_CELL'].append('eg')
awralmod.OUTPUTS['OUTPUTS_CELL'].append('y')
awralmod.OUTPUTS['OUTPUTS_CELL'].append('qg')
awralmod.OUTPUTS['OUTPUTS_CELL'].append('qr')
awralmod.OUTPUTS['OUTPUTS_CELL'].append('qif')
awralmod.OUTPUTS['OUTPUTS_HRU'].append('ei')
awralmod.OUTPUTS['OUTPUTS_HRU'].append('et')
awralmod.OUTPUTS['OUTPUTS_HRU'].append('es')
awralmod.OUTPUTS['OUTPUTS_HRU'].append('fsat')
awralmod.OUTPUTS['OUTPUTS_HRU'].append('fegt')
awralmod.OUTPUTS['OUTPUTS_HRU'].append('us')
awralmod.OUTPUTS['OUTPUTS_HRU'].append('ud')

In [ ]:
awralmod.get_output_mapping()

## 3.1.C. ***Extent*** specification
### The area over which the simulation/calibration/extraction occurs

In [ ]:
# load extents
from awrams.utils import extents

In [ ]:
# default extent - all of Australia
DefExt = extents.get_default_extent()  ## Australia, set as reference extent
DefExt

In [ ]:
# What is used to define this default extent?
sys_settings['DEFAULT_MASK_FILE']

In [ ]:
#### Any rectangular extent
my_extent = DefExt.icoords[-39.5:-44, 143.5: 149]   # Tasmania
my_extent

In [ ]:
#### a single point based on [lat, lon] pair
my_extent  = DefExt.icoords[-34,117]
print(my_extent)
print(my_extent.cell_count)

In [ ]:
##### extents from a shapefile (CATCHMENT_SHAPEFILE contains all calibraito and validation catchments)
from awrams.utils.gis import ShapefileDB
CATCHMENT_SHAPEFILE = sys_settings['DATA_PATHS']['CATCHMENT_SHAPEFILE']
calvalshapefile = ShapefileDB(CATCHMENT_SHAPEFILE)
catchments = calvalshapefile.get_records_df()
# view only the first 5 rows of the shapefile table by using .head()
catchments.head()

In [ ]:
## Create a dictionary with multiple extents
# you can use individual extents from  
cal_dict = {}
cal_catchments = ['204007','421103']
for catchment in cal_catchments:
    cal_dict[catchment] = calvalshapefile.get_extent_by_field('StationID', catchment.zfill(6), parent_extent=DefExt)
cal_dict

## 3.1.D. ***Period*** specification
### Thie time period over which the simulation occurs

In [ ]:
from awrams.utils import datetools as dt

In [ ]:
myperiod = dt.dates('2000', '2010')

In [ ]:
myperiod

In [ ]:
period = dt.dates('dec 2010 - 31 jan 2011')

In [ ]:
period

In [ ]:
dt.dates?

### Demonstrational simulation
#### we will go through this in detail later

In [ ]:
## use the ondemand simulation engine
from awrams.simulation import ondemand

# Change back the path to training data forcing
change_path_to_forcing(input_mapping, path = '/climate/bom_awap/')

# get and instance of the model runner engine - passing in the model (awral and input_map defined earlier)
runner = ondemand.OnDemandSimulator(awralmod,input_mapping) #,omapping=omap.mapping)

# Call the run fun function (within the OnDemandSimulator) over the defined period and extent
r,i = runner.run(period,my_extent,True,False)

import pandas as pd

# Transfer inputs
# transfer climate inputs contained in i
forcing = ('tmin_f','tmax_f','solar_f','precip_f', 'u2t', 'wind_f')
clm = {k:i[k].reshape(-1) for k in forcing}
df_clm = pd.DataFrame(clm,index=period)

# Plot inputs/outputs as timeseries
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize = (16,3))
plt.plot(df_clm)
plt.title('Climate inputs')
plt.legend(df_clm.columns)

plt.figure(figsize = (16,3))
plt.plot(r['etot'],label='etot : Actual evapotranspiration')
plt.plot(r['qtot'],label='qtot: Runoff')
plt.plot(r['e0'],label='e0 : Potential evaporation')
plt.plot(r['dd'],label='dd : Deep drainage')
plt.title('AWRA output water fluxes')
plt.xlabel('Day')
plt.ylabel('Flux [mm]')
plt.legend()
plt.figure(figsize = (16,3))

plt.plot(r['s0'],label='s0 : Top layer [0-10cm]soil moisture')
plt.plot(r['ss'],label='ss : Shallow layer [10-100cm] soil moisture')
plt.plot(r['sd'],label='sd : Deep layer [1 - 6m] soil moisture')
plt.plot(r['sg'],label='sg : Groundwater layer [< 6m] saturated storage')
plt.plot(r['sr'],label='sr : Surface water storage')
plt.xlabel('Day')
plt.ylabel('Storage [mm]')
plt.title('AWRA output water balance stores')
plt.legend()

print('AWRA output variable names r.keys():')
r.keys()

## Now onto The AWRA-CMS Configuration System....

#### [1.3 The AWRA-CMS Configuration System]
[1.3 The AWRA-CMS Configuration System]: 1.3_The_AWRA-CMS_Configuration_System.ipynb